# Lightweight Fine-Tuning Project


* PEFT technique: LoRA
* Model: DistilBERT (distilbert-base-uncased)
* Evaluation approach: Hugging Face Trainer
* Fine-tuning dataset: imdb

In [1]:

%pip install evaluate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U transformers datasets accelerate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import transformers
print(transformers.__version__)

4.57.0


In [4]:
#How to find datasets on HuggingFace Hub
from huggingface_hub import list_datasets
from datasets import load_dataset

datasets_list = list_datasets(limit=20)  # Limit to 20 datasets to avoid rate limit
#From the HuggingFace Hub lets grab a dataset - easy one is the imdb dataset    
print(','.join(dataset.id for dataset in datasets_list))


Agent-Ark/Toucan-1.5M,fka/awesome-chatgpt-prompts,openai/gdpval,Salesforce/Webscale-RL,Jr23xd23/ArabicText-Large,WNJXYK/MATH-Reasoning-Paths,zwhe99/DeepMath-103K,HuggingFaceFW/finepdfs,WNJXYK/MathOdyssey-Reasoning-Paths,WNJXYK/OlympiadBench-Reasoning-Paths,Anthropic/hh-rlhf,google/svq,allenai/WildChat-1M,hotpotqa/hotpot_qa,WNJXYK/AIME_1983_2024-Reasoning-Paths,MTSAIR/MWS-Vision-Bench,WNJXYK/LawQA,omniretarget/OmniRetarget_Dataset,wikimedia/wikipedia,HuggingFaceFW/fineweb


In [5]:

#some datasets have multiple configurations
#we can check the available configurations for a dataset
from datasets import get_dataset_config_names
print(get_dataset_config_names("imdb"))

['plain_text']


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import load_dataset
import numpy as np
import evaluate 

# --- Load Tokenizer and Dataset ---
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
dataset = load_dataset('imdb')

small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
small_test_dataset = dataset["test"].shuffle(seed=42).select(range(1000))

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)



##Check the we tokenized the examples correctly
print(tokenized_train[0])
print(tokenized_test[0])


{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1, 'input_ids': [101, 2045, 2003, 2053, 7189, 2012, 2035, 2090, 3481, 3771, 1998, 6337, 2099, 2021, 1996, 2755, 2008, 2119, 2024, 2610, 2186, 2055, 6355, 6997, 1012, 6337, 2099, 3504, 15594, 2100, 1010, 3481, 3771, 350

In [7]:
# --- Load Model ---
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

# Freeze base model parameters to train only classifier head
for param in model.base_model.parameters():
    param.requires_grad = False

    model.classifier 



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
print(model.classifier)

Linear(in_features=768, out_features=2, bias=True)


In [9]:
# --- Metric function ---
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == labels).mean()}

# --- Data collator ---
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# --- Training Arguments ---
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01
)

# --- Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


# --- Train and Evaluate ---
print("Training the model (classification head only)...")
trainer.train()

print("Evaluating on test set...")
results = trainer.evaluate()

print(f"Evaluation results: {results}")


C:\Users\jessica.jumelles\AppData\Local\Temp\ipykernel_17268\438697938.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training the model (classification head only)...


C:\Users\jessica.jumelles\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


C:\Users\jessica.jumelles\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Evaluating on test set...


C:\Users\jessica.jumelles\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Evaluation results: {'eval_loss': 0.6804912686347961, 'eval_accuracy': 0.61, 'eval_runtime': 37.1188, 'eval_samples_per_second': 26.941, 'eval_steps_per_second': 1.697, 'epoch': 2.0}


In [10]:
# ============================================================
# APPLY PEFT (LoRA) FOR LIGHTWEIGHT FINE-TUNING
# ============================================================

from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)


# --- Create a LoRA configuration ---
# Specify target_modules for DistilBERT attention layers so PEFT/LoRA knows where to apply adapters.
lora_config = LoraConfig(
    task_type="SEQ_CLS",   # sequence classification
    r=8,                   # rank of the LoRA update matrices
    lora_alpha=32,         # scaling factor
    lora_dropout=0.1,      # dropout for LoRA layers
    bias="none",
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"],
)

# --- Wrap the pretrained model with the LoRA adapter ---

peft_model = get_peft_model(model, lora_config)

# Check trainable parameters (to confirm LoRA is applied)
peft_model.print_trainable_parameters()

# ============================================================
#  TRAIN THE PEFT (LoRA) MODEL
# ============================================================

# Reuse same training arguments but change output directory
lora_training_args = TrainingArguments(
    output_dir="./lora_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,   # LoRA can use a slightly higher LR
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
)

# Create Trainer for LoRA model
lora_trainer = Trainer(
    model=peft_model,
    args=lora_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Fine-tuning the model with LoRA adapters...")
lora_trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 887,042 || all params: 67,842,052 || trainable%: 1.3075


C:\Users\jessica.jumelles\AppData\Local\Temp\ipykernel_17268\2147321303.py:50: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  lora_trainer = Trainer(


Fine-tuning the model with LoRA adapters...


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.438738,0.810000
2,No log,0.419289,0.823000


C:\Users\jessica.jumelles\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=126, training_loss=0.5073725079733228, metrics={'train_runtime': 368.6446, 'train_samples_per_second': 5.425, 'train_steps_per_second': 0.342, 'total_flos': 67596195840000.0, 'train_loss': 0.5073725079733228, 'epoch': 2.0})

In [11]:
 #SAVE THE PEFT MODEL
# ============================================================

save_directory = "./peft_lora_model"
peft_model.save_pretrained(save_directory)
print(f"LoRA fine-tuned model saved to: {save_directory}")


LoRA fine-tuned model saved to: ./peft_lora_model


In [12]:
#LOAD AND EVALUATE THE FINE-TUNED MODEL
# ============================================================


base_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

# Load Saved PEFT Model 
peft_model_loaded = AutoPeftModelForSequenceClassification.from_pretrained(save_directory)

# Create Trainer for evaluation
eval_trainer = Trainer(
    model=peft_model_loaded,
    args=training_args,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Evaluating the LoRA fine-tuned model...")
final_results = eval_trainer.evaluate()
print(f"Fine-tuned model accuracy: {final_results['eval_accuracy']:.4f}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\jessica.jumelles\AppData\Local\Temp\ipykernel_17268\3205574363.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(
C:\Users\jessica.jumelles\AppData\Roaming\Python\Python

Evaluating the LoRA fine-tuned model...


Fine-tuned model accuracy: 0.8230


In [13]:
#compare results
print(f"Base model accuracy: {results['eval_accuracy']:.4f}")
print(f"Fine-tuned model accuracy: {final_results['eval_accuracy']:.4f}")

Base model accuracy: 0.6100
Fine-tuned model accuracy: 0.8230
